##CV features

In [313]:
import re
import gensim 
import numpy as np
from tqdm import tqdm 
from functools import reduce
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from gensim.models import FastText
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import seaborn as sns
import spacy  # For preprocessing
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
from sklearn.model_selection import train_test_split

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [268]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [269]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/NLP/df_final_cv.csv")

In [270]:
df

,Unnamed: 0,status,description,username,fbid,connected_fb_page,profile_pic_url,full_name,is_private,category_name,...,input_stop_id,warning,error,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['text', 'font', 'graphics', 'screenshot', 'gr...","[0.9980798959732056, 0.9481294751167297, 0.881...","(0.788235294117647, 0.00784313725490196, 0.007...",0,"[(1.0, 0.7529411764705882, 0.796078431372549)]","(1.0, 0.7529411764705882, 0.796078431372549)","(1.0, 0.7529411764705882, 0.796078431372549)"
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['dessert', 'baked goods', 'baking', 'snack', ...","[0.9897554516792297, 0.987897515296936, 0.9828...","(0.6901960784313725, 0.14901960784313725, 0.10...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.0, 0.0, 0.0)"
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['food', 'fruit', 'baked goods', 'breakfast', ...","[0.9808361530303955, 0.9546540379524231, 0.947...","(0.6588235294117647, 0.4392156862745098, 0.141...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.6470588235294118, 0.16470588235294117, 0.16..."
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['food', 'bread', 'fast food', 'bun', 'america...","[0.9958561658859253, 0.981575608253479, 0.9799...","(0.24313725490196078, 0.12549019607843137, 0.0...",0,"[(0.6470588235294118, 0.16470588235294117, 0.1...","(0.6470588235294118, 0.16470588235294117, 0.16...","(0.6470588235294118, 0.16470588235294117, 0.16..."
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['text', 'font', 'design', 'yellow', 'graphics...","[0.9993002414703369, 0.9640201926231384, 0.951...","(0.792156862745098, 0.7372549019607844, 0.0039...",0,"[(1.0, 1.0, 0.0)]","(1.0, 1.0, 0.0)","(1.0, 1.0, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,88511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['text', 'food', 'snack']","[0.9999812841415405, 0.9580191373825073, 0.915...","(0.6588235294117647, 0.00784313725490196, 0.71...",0,"[(0.5019607843137255, 0.0, 0.5019607843137255)...","(0.5019607843137255, 0.0, 0.5019607843137255)","(0.5019607843137255, 0.5019607843137255, 0.501..."
88512,88512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['text', 'poster', 'cartoon']","[0.9999863505363464, 0.9184243679046631, 0.910...","(0.01568627450980392, 0.5098039215686274, 0.78...",0,"[(1.0, 1.0, 1.0)]","(1.0, 1.0, 1.0)","(1.0, 1.0, 1.0)"
88513,88513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"['text', 'yellow', 'packaging and labeling', '...","[0.9997624158859253, 0.9444339871406555, 0.854...","(0.01568627450980392, 0.2235294117647059, 0.43...",0,"[(1.0, 1.0, 0.0), (0.0, 0.0, 1.0)]","(0.0, 0.0, 1.0)","(1.0, 1.0, 0.0)"
88514,88514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Crawler error: Redirected to login page,"['sport', 'person', 'barbell', 'physical fitne...","[0.9959811568260193, 0.9808116555213928, 0.971...","(0.6862745098039216, 0.5215686274509804, 0.109...",0,"[(1.0, 1.0, 1.0)]","(1.0, 1.0, 1.0)","(1.0, 1.0, 1.0)"


In [271]:
df.drop(df.columns.difference(['tags','confidence_score','accent_color','is_bw','dominant_colors','bg_color','fore_color','likes']), 1, inplace=True)

In [272]:
#calculating missing values in the dataset
missing_values = df.isnull().sum()
missing_per = (missing_values/df.shape[0])*100
missing_table = pd.concat([missing_values,missing_per], axis=1, ignore_index=True) 
missing_table.rename(columns={0:'Total Missing Values',1:'Missing %'}, inplace=True)
missing_table

,Total Missing Values,Missing %
likes,2,0.002259
tags,0,0.000000
confidence_score,0,0.000000
accent_color,0,0.000000
is_bw,0,0.000000
dominant_colors,0,0.000000
bg_color,0,0.000000
fore_color,0,0.000000


In [273]:
#fill the missing values with zero

df['likes'] = df['likes'].fillna(0)

In [274]:
df1=df[:]

In [275]:
REPLACE_BY_SPACE_RE = re.compile('[^a-zA-Z_0-9,.]')

def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()


def replace_br(text: str) -> str:
    """
    Replaces br characters
    """

    return text.replace("'", "")


def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)


from typing import List, Callable, Dict

PREPROCESSING_PIPELINE = [
                          replace_special_characters,
                          strip_text
                          ]



def text_prepare(text: str, filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """
    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE
    return reduce(lambda txt, f: f(txt), filter_methods, text)

# Pre-processing

print('Pre-processing text...')
print('[Debug] Before:\n{}'.format(df1.tags[:10]))

L=['tags','confidence_score','accent_color','dominant_colors','bg_color','fore_color']

A=['tags']

for i in L:
  df1[i]=df1[i].apply(lambda txt: text_prepare(str(txt)[1:-1]))



print('[Debug] After:\n{}'.format(df1.tags[:10]))
print()

print("Pre-processing completed!")

Pre-processing text...
[Debug] Before:
0    ['text', 'font', 'graphics', 'screenshot', 'gr...
1    ['dessert', 'baked goods', 'baking', 'snack', ...
2    ['food', 'fruit', 'baked goods', 'breakfast', ...
3    ['food', 'bread', 'fast food', 'bun', 'america...
4    ['text', 'font', 'design', 'yellow', 'graphics...
5    ['snack', 'baked goods', 'baking', 'dessert', ...
6    ['dessert', 'baked goods', 'baking', 'sweetnes...
7    ['text', 'food', 'shelf', 'countertop', 'indoo...
8    ['text', 'table', 'food', 'bakery', 'clothing'...
9    ['baked goods', 'sweetness', 'food', 'baking',...
Name: tags, dtype: object
[Debug] After:
0    text ,  font ,  graphics ,  screenshot ,  grap...
1    dessert ,  baked goods ,  baking ,  snack ,  f...
2    food ,  fruit ,  baked goods ,  breakfast ,  c...
3    food ,  bread ,  fast food ,  bun ,  american ...
4    text ,  font ,  design ,  yellow ,  graphics ,...
5    snack ,  baked goods ,  baking ,  dessert ,  b...
6    dessert ,  baked goods ,  baking , 

In [276]:
df1

,likes,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,29.0,"text , font , graphics , screenshot , grap...","0.9980798959732056, 0.9481294751167297, 0.8818...","0.788235294117647, 0.00784313725490196, 0.0078...",0,"1.0, 0.7529411764705882, 0.796078431372549","1.0, 0.7529411764705882, 0.796078431372549","1.0, 0.7529411764705882, 0.796078431372549"
1,44.0,"dessert , baked goods , baking , snack , f...","0.9897554516792297, 0.987897515296936, 0.98287...","0.6901960784313725, 0.14901960784313725, 0.105...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.0, 0.0, 0.0"
2,209.0,"food , fruit , baked goods , breakfast , c...","0.9808361530303955, 0.9546540379524231, 0.9477...","0.6588235294117647, 0.4392156862745098, 0.1411...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164..."
3,95.0,"food , bread , fast food , bun , american ...","0.9958561658859253, 0.981575608253479, 0.97992...","0.24313725490196078, 0.12549019607843137, 0.07...",0,"0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164...","0.6470588235294118, 0.16470588235294117, 0.164..."
4,101.0,"text , font , design , yellow , graphics ,...","0.9993002414703369, 0.9640201926231384, 0.9513...","0.792156862745098, 0.7372549019607844, 0.00392...",0,"1.0, 1.0, 0.0","1.0, 1.0, 0.0","1.0, 1.0, 0.0"
...,...,...,...,...,...,...,...,...
88511,8.0,"text , food , snack","0.9999812841415405, 0.9580191373825073, 0.9157...","0.6588235294117647, 0.00784313725490196, 0.713...",0,"0.5019607843137255, 0.0, 0.5019607843137255 , ...","0.5019607843137255, 0.0, 0.5019607843137255","0.5019607843137255, 0.5019607843137255, 0.5019..."
88512,9.0,"text , poster , cartoon","0.9999863505363464, 0.9184243679046631, 0.9104...","0.01568627450980392, 0.5098039215686274, 0.780...",0,"1.0, 1.0, 1.0","1.0, 1.0, 1.0","1.0, 1.0, 1.0"
88513,11.0,"text , yellow , packaging and labeling , fo...","0.9997624158859253, 0.9444339871406555, 0.8540...","0.01568627450980392, 0.2235294117647059, 0.439...",0,"1.0, 1.0, 0.0 , 0.0, 0.0, 1.0","0.0, 0.0, 1.0","1.0, 1.0, 0.0"
88514,0.0,"sport , person , barbell , physical fitness...","0.9959811568260193, 0.9808116555213928, 0.9714...","0.6862745098039216, 0.5215686274509804, 0.1098...",0,"1.0, 1.0, 1.0","1.0, 1.0, 1.0","1.0, 1.0, 1.0"


In [277]:
# for i in df1['confidence_score']:

def convert_str2int(i):
  n = []
  y = i.split(', ')
  if len(y[0]) == 0:
    return []
  for j in range(len(y)):
    n.append(float(y[j]))
  return n

df1['confidence_score'] = df1['confidence_score'].apply(convert_str2int)
df1['accent_color'] = df1['accent_color'].apply(convert_str2int)
df1['dominant_colors'] = df1['dominant_colors'].apply(convert_str2int)
df1['bg_color'] = df1['bg_color'].apply(convert_str2int)
df1['fore_color'] = df1['fore_color'].apply(convert_str2int)

In [278]:
df1

,likes,tags,confidence_score,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,29.0,"text , font , graphics , screenshot , grap...","[0.9980798959732056, 0.9481294751167297, 0.881...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]"
1,44.0,"dessert , baked goods , baking , snack , f...","[0.9897554516792297, 0.987897515296936, 0.9828...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]"
2,209.0,"food , fruit , baked goods , breakfast , c...","[0.9808361530303955, 0.9546540379524231, 0.947...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16..."
3,95.0,"food , bread , fast food , bun , american ...","[0.9958561658859253, 0.981575608253479, 0.9799...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16..."
4,101.0,"text , font , design , yellow , graphics ,...","[0.9993002414703369, 0.9640201926231384, 0.951...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...,...
88511,8.0,"text , food , snack","[0.9999812841415405, 0.9580191373825073, 0.915...","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255, ...","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501..."
88512,9.0,"text , poster , cartoon","[0.9999863505363464, 0.9184243679046631, 0.910...","[0.01568627450980392, 0.5098039215686274, 0.78...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]"
88513,11.0,"text , yellow , packaging and labeling , fo...","[0.9997624158859253, 0.9444339871406555, 0.854...","[0.01568627450980392, 0.2235294117647059, 0.43...",0,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[1.0, 1.0, 0.0]"
88514,0.0,"sport , person , barbell , physical fitness...","[0.9959811568260193, 0.9808116555213928, 0.971...","[0.6862745098039216, 0.5215686274509804, 0.109...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]"


In [279]:
def convert_str2list(i):
  n = []
  y = i.split(', ')
  for j in range(len(y)):
    n.append(str(y[j]))
  return n
df1['tags'] = df['tags'].apply(convert_str2list)
df1['tags'][0]

['text ',
 ' font ',
 ' graphics ',
 ' screenshot ',
 ' graphic design ',
 ' design ',
 ' typography']

In [280]:
df1['confidence_score'][6]

[0.9938297271728516,
 0.9784528613090515,
 0.977806806564331,
 0.966992974281311,
 0.9616279602050781,
 0.9515609741210938,
 0.9280351400375366,
 0.912075936794281,
 0.9083282947540283,
 0.9079588651657104,
 0.8951475620269775,
 0.8874362707138062,
 0.8836393356323242,
 0.8746436834335327,
 0.8713182210922241,
 0.8668385744094849,
 0.8426526784896851,
 0.786042332649231,
 0.5681669116020203]

In [281]:
def trim_lowConfidence_tags(tag_column):
  for scoreList in df1['confidence_score']:
    for score_index in range(len(scoreList)):
      if scoreList[score_index]<0.5:
        return tag_column[:score_index]

df1['tags'] = df1['tags'].apply(trim_lowConfidence_tags)  

In [282]:
df1.drop('confidence_score',1,inplace=True)

In [283]:
df1

,likes,tags,accent_color,is_bw,dominant_colors,bg_color,fore_color
0,29.0,"[text , font , graphics , screenshot , gra...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]"
1,44.0,"[dessert , baked goods , baking , snack , ...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]"
2,209.0,"[food , fruit , baked goods , breakfast , ...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16..."
3,95.0,"[food , bread , fast food , bun , american...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16..."
4,101.0,"[text , font , design , yellow , graphics ...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...
88511,8.0,"[text , food , snack]","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255, ...","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501..."
88512,9.0,"[text , poster , cartoon]","[0.01568627450980392, 0.5098039215686274, 0.78...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]"
88513,11.0,"[text , yellow , packaging and labeling , f...","[0.01568627450980392, 0.2235294117647059, 0.43...",0,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[1.0, 1.0, 0.0]"
88514,0.0,"[sport , person , barbell , physical fitnes...","[0.6862745098039216, 0.5215686274509804, 0.109...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]"


In [284]:
df1['dominant_colors'][:3]

0         [1.0, 0.7529411764705882, 0.796078431372549]
1    [0.6470588235294118, 0.16470588235294117, 0.16...
2    [0.6470588235294118, 0.16470588235294117, 0.16...
Name: dominant_colors, dtype: object

In [285]:
def separate_column(column):
  if len(column)>0:
    return column[0]
  else:
    return []

df1['dominant_colors_R'] = df1['dominant_colors'].apply(separate_column)  
df1['accent_color_R'] = df1['accent_color'].apply(separate_column)  
df1['bg_color_R'] = df1['bg_color'].apply(separate_column)  
df1['fore_color_R'] = df1['fore_color'].apply(separate_column)  

In [286]:
def separate_column(column):
  if len(column)>0:
    return column[1]
  else:
    return []

df1['dominant_colors_G'] = df1['dominant_colors'].apply(separate_column)  
df1['accent_color_G'] = df1['accent_color'].apply(separate_column)  
df1['bg_color_G'] = df1['bg_color'].apply(separate_column)  
df1['fore_color_G'] = df1['fore_color'].apply(separate_column)  

In [287]:
def separate_column(column):
  if len(column)>0:
    return column[2]
  else:
    return []

df1['dominant_colors_B'] = df1['dominant_colors'].apply(separate_column)  
df1['accent_color_B'] = df1['accent_color'].apply(separate_column)  
df1['bg_color_B'] = df1['bg_color'].apply(separate_column)  
df1['fore_color_B'] = df1['fore_color'].apply(separate_column)  

In [288]:
df1

,likes,tags,accent_color,is_bw,dominant_colors,bg_color,fore_color,dominant_colors_R,accent_color_R,bg_color_R,fore_color_R,dominant_colors_G,accent_color_G,bg_color_G,fore_color_G,dominant_colors_B,accent_color_B,bg_color_B,fore_color_B
0,29.0,"[text , font , graphics , screenshot , gra...","[0.788235294117647, 0.00784313725490196, 0.007...",0,"[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]","[1.0, 0.7529411764705882, 0.796078431372549]",1.0,0.788235,1.000000,1.000000,0.752941,0.007843,0.752941,0.752941,0.796078,0.007843,0.796078,0.796078
1,44.0,"[dessert , baked goods , baking , snack , ...","[0.6901960784313725, 0.14901960784313725, 0.10...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.0, 0.0, 0.0]",0.647059,0.690196,0.647059,0.000000,0.164706,0.149020,0.164706,0.000000,0.164706,0.105882,0.164706,0.000000
2,209.0,"[food , fruit , baked goods , breakfast , ...","[0.6588235294117647, 0.4392156862745098, 0.141...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",0.647059,0.658824,0.647059,0.647059,0.164706,0.439216,0.164706,0.164706,0.164706,0.141176,0.164706,0.164706
3,95.0,"[food , bread , fast food , bun , american...","[0.24313725490196078, 0.12549019607843137, 0.0...",0,"[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...","[0.6470588235294118, 0.16470588235294117, 0.16...",0.647059,0.243137,0.647059,0.647059,0.164706,0.125490,0.164706,0.164706,0.164706,0.070588,0.164706,0.164706
4,101.0,"[text , font , design , yellow , graphics ...","[0.792156862745098, 0.7372549019607844, 0.0039...",0,"[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]","[1.0, 1.0, 0.0]",1.0,0.792157,1.000000,1.000000,1.0,0.737255,1.000000,1.000000,0.0,0.003922,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,8.0,"[text , food , snack]","[0.6588235294117647, 0.00784313725490196, 0.71...",0,"[0.5019607843137255, 0.0, 0.5019607843137255, ...","[0.5019607843137255, 0.0, 0.5019607843137255]","[0.5019607843137255, 0.5019607843137255, 0.501...",0.501961,0.658824,0.501961,0.501961,0.0,0.007843,0.000000,0.501961,0.501961,0.713725,0.501961,0.501961
88512,9.0,"[text , poster , cartoon]","[0.01568627450980392, 0.5098039215686274, 0.78...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",1.0,0.015686,1.000000,1.000000,1.0,0.509804,1.000000,1.000000,1.0,0.780392,1.000000,1.000000
88513,11.0,"[text , yellow , packaging and labeling , f...","[0.01568627450980392, 0.2235294117647059, 0.43...",0,"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[1.0, 1.0, 0.0]",1.0,0.015686,0.000000,1.000000,1.0,0.223529,0.000000,1.000000,0.0,0.439216,1.000000,0.000000
88514,0.0,"[sport , person , barbell , physical fitnes...","[0.6862745098039216, 0.5215686274509804, 0.109...",0,"[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]","[1.0, 1.0, 1.0]",1.0,0.686275,1.000000,1.000000,1.0,0.521569,1.000000,1.000000,1.0,0.109804,1.000000,1.000000


In [289]:
dropped = ['accent_color', 'dominant_colors', 'bg_color','fore_color']
data = df1.drop(dropped,axis=1)

In [290]:
data

,likes,tags,is_bw,dominant_colors_R,accent_color_R,bg_color_R,fore_color_R,dominant_colors_G,accent_color_G,bg_color_G,fore_color_G,dominant_colors_B,accent_color_B,bg_color_B,fore_color_B
0,29.0,"[text , font , graphics , screenshot , gra...",0,1.0,0.788235,1.000000,1.000000,0.752941,0.007843,0.752941,0.752941,0.796078,0.007843,0.796078,0.796078
1,44.0,"[dessert , baked goods , baking , snack , ...",0,0.647059,0.690196,0.647059,0.000000,0.164706,0.149020,0.164706,0.000000,0.164706,0.105882,0.164706,0.000000
2,209.0,"[food , fruit , baked goods , breakfast , ...",0,0.647059,0.658824,0.647059,0.647059,0.164706,0.439216,0.164706,0.164706,0.164706,0.141176,0.164706,0.164706
3,95.0,"[food , bread , fast food , bun , american...",0,0.647059,0.243137,0.647059,0.647059,0.164706,0.125490,0.164706,0.164706,0.164706,0.070588,0.164706,0.164706
4,101.0,"[text , font , design , yellow , graphics ...",0,1.0,0.792157,1.000000,1.000000,1.0,0.737255,1.000000,1.000000,0.0,0.003922,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,8.0,"[text , food , snack]",0,0.501961,0.658824,0.501961,0.501961,0.0,0.007843,0.000000,0.501961,0.501961,0.713725,0.501961,0.501961
88512,9.0,"[text , poster , cartoon]",0,1.0,0.015686,1.000000,1.000000,1.0,0.509804,1.000000,1.000000,1.0,0.780392,1.000000,1.000000
88513,11.0,"[text , yellow , packaging and labeling , f...",0,1.0,0.015686,0.000000,1.000000,1.0,0.223529,0.000000,1.000000,0.0,0.439216,1.000000,0.000000
88514,0.0,"[sport , person , barbell , physical fitnes...",0,1.0,0.686275,1.000000,1.000000,1.0,0.521569,1.000000,1.000000,1.0,0.109804,1.000000,1.000000


In [291]:
data2=data.drop('tags',1)

In [292]:
data2

,likes,is_bw,dominant_colors_R,accent_color_R,bg_color_R,fore_color_R,dominant_colors_G,accent_color_G,bg_color_G,fore_color_G,dominant_colors_B,accent_color_B,bg_color_B,fore_color_B
0,29.0,0,1.0,0.788235,1.000000,1.000000,0.752941,0.007843,0.752941,0.752941,0.796078,0.007843,0.796078,0.796078
1,44.0,0,0.647059,0.690196,0.647059,0.000000,0.164706,0.149020,0.164706,0.000000,0.164706,0.105882,0.164706,0.000000
2,209.0,0,0.647059,0.658824,0.647059,0.647059,0.164706,0.439216,0.164706,0.164706,0.164706,0.141176,0.164706,0.164706
3,95.0,0,0.647059,0.243137,0.647059,0.647059,0.164706,0.125490,0.164706,0.164706,0.164706,0.070588,0.164706,0.164706
4,101.0,0,1.0,0.792157,1.000000,1.000000,1.0,0.737255,1.000000,1.000000,0.0,0.003922,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,8.0,0,0.501961,0.658824,0.501961,0.501961,0.0,0.007843,0.000000,0.501961,0.501961,0.713725,0.501961,0.501961
88512,9.0,0,1.0,0.015686,1.000000,1.000000,1.0,0.509804,1.000000,1.000000,1.0,0.780392,1.000000,1.000000
88513,11.0,0,1.0,0.015686,0.000000,1.000000,1.0,0.223529,0.000000,1.000000,0.0,0.439216,1.000000,0.000000
88514,0.0,0,1.0,0.686275,1.000000,1.000000,1.0,0.521569,1.000000,1.000000,1.0,0.109804,1.000000,1.000000


In [293]:
data['tags']=data['tags'].apply(lambda txt: text_prepare(str(txt)))

In [294]:
tokenized_tag = data['tags'].apply(lambda x: x.split())
tokenized_tag[0]

['text',
 ',',
 'font',
 ',',
 'graphics',
 ',',
 'screenshot',
 ',',
 'graphic',
 'design',
 ',',
 'design',
 ',',
 'typography']

##Word2vec

### Training for skip gram

In [295]:
model_w2v_skipgram = gensim.models.Word2Vec(tokenized_tag,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
)
model_w2v_skipgram.train(tokenized_tag, total_examples= len(tokenized_tag), epochs=20)
model_w2v_skipgram.save("/content/drive/MyDrive/NLP/Models/200_word2vec_skipgram_.model")

INFO - 01:31:53: collecting all words and their counts
INFO - 01:31:53: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 01:31:53: PROGRESS: at sentence #10000, processed 204860 words, keeping 2339 word types
INFO - 01:31:54: PROGRESS: at sentence #20000, processed 415054 words, keeping 2816 word types
INFO - 01:31:54: PROGRESS: at sentence #30000, processed 622777 words, keeping 3089 word types
INFO - 01:31:54: PROGRESS: at sentence #40000, processed 834536 words, keeping 3287 word types
INFO - 01:31:54: PROGRESS: at sentence #50000, processed 1041326 words, keeping 3425 word types
INFO - 01:31:54: PROGRESS: at sentence #60000, processed 1253866 words, keeping 3563 word types
INFO - 01:31:54: PROGRESS: at sentence #70000, processed 1462839 words, keeping 3664 word types
INFO - 01:31:54: PROGRESS: at sentence #80000, processed 1672957 words, keeping 3777 word types
INFO - 01:31:54: collected 3831 word types from a corpus of 1850422 raw words and 88516 sentences


In [ ]:
model_w2v_skipgram.wv.most_similar("Gastroenteritis")

### Training for CBOW 

In [296]:
model_w2v_CBOW = gensim.models.Word2Vec(tokenized_tag,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 0, # 0 for CBOW model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
)
model_w2v_CBOW.train(tokenized_tag, total_examples= len(tokenized_tag), epochs=20)
model_w2v_CBOW.save("/content/drive/MyDrive/NLP/Models/200_word2vec_CBOW_.model")

INFO - 01:34:48: collecting all words and their counts
INFO - 01:34:48: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 01:34:48: PROGRESS: at sentence #10000, processed 204860 words, keeping 2339 word types
INFO - 01:34:48: PROGRESS: at sentence #20000, processed 415054 words, keeping 2816 word types
INFO - 01:34:48: PROGRESS: at sentence #30000, processed 622777 words, keeping 3089 word types
INFO - 01:34:48: PROGRESS: at sentence #40000, processed 834536 words, keeping 3287 word types
INFO - 01:34:48: PROGRESS: at sentence #50000, processed 1041326 words, keeping 3425 word types
INFO - 01:34:49: PROGRESS: at sentence #60000, processed 1253866 words, keeping 3563 word types
INFO - 01:34:49: PROGRESS: at sentence #70000, processed 1462839 words, keeping 3664 word types
INFO - 01:34:49: PROGRESS: at sentence #80000, processed 1672957 words, keeping 3777 word types
INFO - 01:34:49: collected 3831 word types from a corpus of 1850422 raw words and 88516 sentences


In [ ]:
model_w2v_CBOW.wv.most_similar("Gastroenteritis")

##FastText

In [315]:
modelFastTextSkipGram = FastText(tokenized_tag, 
                     size=200, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=1  # 1 for Skipgram model
                     )

modelFastTextSkipGram.build_vocab(tokenized_tag, update=True)
modelFastTextSkipGram.train(tokenized_tag, total_examples= len(tokenized_tag), epochs=20)
modelFastTextSkipGram.save("/content/drive/MyDrive/NLP/Models/200_fastText_Skipgram_.model")

INFO - 03:02:39: collecting all words and their counts
INFO - 03:02:39: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 03:02:39: PROGRESS: at sentence #10000, processed 204860 words, keeping 2339 word types
INFO - 03:02:39: PROGRESS: at sentence #20000, processed 415054 words, keeping 2816 word types
INFO - 03:02:39: PROGRESS: at sentence #30000, processed 622777 words, keeping 3089 word types
INFO - 03:02:39: PROGRESS: at sentence #40000, processed 834536 words, keeping 3287 word types
INFO - 03:02:39: PROGRESS: at sentence #50000, processed 1041326 words, keeping 3425 word types
INFO - 03:02:39: PROGRESS: at sentence #60000, processed 1253866 words, keeping 3563 word types
INFO - 03:02:39: PROGRESS: at sentence #70000, processed 1462839 words, keeping 3664 word types
INFO - 03:02:39: PROGRESS: at sentence #80000, processed 1672957 words, keeping 3777 word types
INFO - 03:02:39: collected 3831 word types from a corpus of 1850422 raw words and 88516 sentences


In [321]:
modelFastTextSkipGram.wv.most_similar("pizza")

[('flatbread', 0.7164261937141418),
 ('casserole', 0.6756445169448853),
 ('italian', 0.6681921482086182),
 ('pepperoni', 0.6635985970497131),
 ('manakish', 0.6345478892326355),
 ('sicilian', 0.6295322775840759),
 ('california', 0.626754641532898),
 ('quiche', 0.6123692393302917),
 ('cassolette', 0.5972737669944763),
 ('macaroni', 0.5769228935241699)]

In [ ]:
modelFastTextCBOW = FastText(tokenized_tag, 
                     size=200, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=0  # 0 for CBOW model
                     )
modelFastTextCBOW.build_vocab(tokenized_tag, update=True)
modelFastTextCBOW.train(tokenized_tag, total_examples= len(tokenized_tag), epochs=20)
modelFastTextCBOW.save("/content/drive/MyDrive/NLP/Models/200_fastText_CBOW_.model")

INFO - 03:10:49: collecting all words and their counts
INFO - 03:10:49: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 03:10:49: PROGRESS: at sentence #10000, processed 204860 words, keeping 2339 word types
INFO - 03:10:49: PROGRESS: at sentence #20000, processed 415054 words, keeping 2816 word types
INFO - 03:10:49: PROGRESS: at sentence #30000, processed 622777 words, keeping 3089 word types
INFO - 03:10:49: PROGRESS: at sentence #40000, processed 834536 words, keeping 3287 word types
INFO - 03:10:49: PROGRESS: at sentence #50000, processed 1041326 words, keeping 3425 word types
INFO - 03:10:49: PROGRESS: at sentence #60000, processed 1253866 words, keeping 3563 word types
INFO - 03:10:49: PROGRESS: at sentence #70000, processed 1462839 words, keeping 3664 word types
INFO - 03:10:49: PROGRESS: at sentence #80000, processed 1672957 words, keeping 3777 word types
INFO - 03:10:49: collected 3831 word types from a corpus of 1850422 raw words and 88516 sentences


In [ ]:
modelFastTextSkipGram.wv.most_similar("Gastroenteritis")

In [297]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v_skipgram[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [298]:
wordvec_arrays = np.zeros((len(tokenized_tag), 200)) 
for i in range(len(tokenized_tag)):
    wordvec_arrays[i,:] = word_vector(tokenized_tag[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(88516, 200)

In [299]:
dataset_word2vec = pd.DataFrame(np.hstack((wordvec_df,data2)))

In [300]:
dataset_word2vec[200]

0         29.0
1         44.0
2        209.0
3         95.0
4        101.0
         ...  
88511      8.0
88512      9.0
88513     11.0
88514      0.0
88515      0.0
Name: 200, Length: 88516, dtype: object

##doc2vec

In [301]:
from gensim.models.doc2vec import LabeledSentence

def add_label(twt):
    output = []
    for i, s in zip(twt.index, twt):
        output.append(LabeledSentence(s, ["tags_" + str(i)]))
    return output

labeled_tags = add_label(tokenized_tag) # label all the tweets

### doc2vec

In [302]:
model_d2vec = gensim.models.Doc2Vec(dm=1, # dm = 1 for ‘distributed memory’ model
                                  dm_mean=1, # dm_mean = 1 for using mean of the context word vectors
                                  vector_size=200, # no. of desired features
                                  window=5, # width of the context window                                  
                                  negative=7, # if > 0 then negative sampling will be used
                                  min_count=5, # Ignores all words with total frequency lower than 5.                                  
                                  workers=32, # no. of cores                                  
                                  alpha=0.1, # learning rate                                  
                                  seed = 23, # for reproducibility
                                 ) 
model_d2vec.build_vocab([i for i in tqdm(labeled_tags)])
model_d2vec.train(labeled_tags, total_examples= len(tokenized_tag), epochs=15)
model_d2vec.save("/content/drive/MyDrive/NLP/Models/200_doc2vec_skipgram_.model")

100%|██████████| 88516/88516 [00:00<00:00, 2452847.60it/s]
INFO - 01:36:13: collecting all words and their counts
INFO - 01:36:13: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO - 01:36:13: PROGRESS: at example #10000, processed 204860 words (2366772/s), 2339 word types, 10000 tags
INFO - 01:36:14: PROGRESS: at example #20000, processed 415054 words (2755716/s), 2816 word types, 20000 tags
INFO - 01:36:14: PROGRESS: at example #30000, processed 622777 words (3071279/s), 3089 word types, 30000 tags
INFO - 01:36:14: PROGRESS: at example #40000, processed 834536 words (2915132/s), 3287 word types, 40000 tags
INFO - 01:36:14: PROGRESS: at example #50000, processed 1041326 words (3042903/s), 3425 word types, 50000 tags
INFO - 01:36:14: PROGRESS: at example #60000, processed 1253866 words (3105562/s), 3563 word types, 60000 tags
INFO - 01:36:14: PROGRESS: at example #70000, processed 1462839 words (3171435/s), 3664 word types, 70000 tags
INFO - 01:36:14: PROGRESS

###doc2vec CBOW

In [303]:
docvec_arrays = np.zeros((len(tokenized_tag), 200)) 
for i in range(len(df1)):
    docvec_arrays[i,:] = model_d2vec.docvecs[i].reshape((1,200))    
docvec_df = pd.DataFrame(docvec_arrays) 
docvec_df.shape

(88516, 200)

In [304]:
dataset_doc2vec = pd.DataFrame(np.hstack((docvec_df,data2)))
dataset_doc2vec

,0,1,2,3,4,5,6,7,8,9,...,204,205,206,207,208,209,210,211,212,213
0,-0.107494,0.038917,-0.074278,-0.025017,-0.056794,0.019376,0.03275,-0.044627,-0.171654,0.079349,...,1.0,1.0,0.752941,0.007843,0.752941,0.752941,0.796078,0.007843,0.796078,0.796078
1,-0.07192,0.162235,-0.127017,-0.262732,-0.10456,-0.144408,-0.000265,0.091538,-0.197478,-0.030604,...,0.647059,0.0,0.164706,0.14902,0.164706,0.0,0.164706,0.105882,0.164706,0.0
2,-0.02308,-0.297094,-0.092259,-0.316978,0.204423,-0.178576,0.085651,-0.260737,0.102897,0.05907,...,0.647059,0.647059,0.164706,0.439216,0.164706,0.164706,0.164706,0.141176,0.164706,0.164706
3,-0.141804,0.221588,-0.04923,-0.37988,-0.231211,-0.204855,0.029463,-0.282709,0.012473,-0.015478,...,0.647059,0.647059,0.164706,0.12549,0.164706,0.164706,0.164706,0.070588,0.164706,0.164706
4,-0.2153,0.067726,-0.216385,-0.047474,0.122216,-0.053014,-0.195862,-0.208554,0.361923,0.047828,...,1.0,1.0,1.0,0.737255,1.0,1.0,0.0,0.003922,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88511,-0.113545,0.203361,-0.08468,-0.210903,0.063462,-0.110391,0.029032,0.029656,0.003247,0.13355,...,0.501961,0.501961,0.0,0.007843,0.0,0.501961,0.501961,0.713725,0.501961,0.501961
88512,0.019931,0.173275,-0.099715,-0.103696,0.162777,-0.112817,-0.001192,0.177968,0.049982,0.051011,...,1.0,1.0,1.0,0.509804,1.0,1.0,1.0,0.780392,1.0,1.0
88513,-0.198902,0.138828,-0.358119,0.197175,0.028369,-0.30766,0.001995,0.029515,-0.067311,0.093603,...,0.0,1.0,1.0,0.223529,0.0,1.0,0.0,0.439216,1.0,0.0
88514,-0.026813,0.157102,-0.097433,-0.105122,-0.159692,-0.040798,-0.094521,0.254486,-0.090869,0.174848,...,1.0,1.0,1.0,0.521569,1.0,1.0,1.0,0.109804,1.0,1.0


## Train test split

In [305]:
y=dataset_word2vec[200]
X1=dataset_word2vec.drop(200,1)

In [306]:
x_train, x_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=2022)

###XGBoost model

In [307]:
#XGBoost hyper-parameter tuning
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

In [312]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
        objective = 'reg:squarederror',
        colsample_bytree = 0.5,
        learning_rate = 0.05,
        max_depth = 6,
        min_child_weight = 1,
        n_estimators = 1000,
        subsample = 0.7)

%time xgb_model.fit(list(x_train), y_train, early_stopping_rounds=5, verbose=False).ravel()

y_pred_xgb = xgb_model.predict(x_test)

XGBoostError: ignored

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print("RMSE: %f" % (rmse))
R2=r2_score(y_test,y_pred_xgb)
print("R2 Score: %f" % (R2))
print()

In [ ]:
def plot_feature_importance(importance,names,model_type):

#Create arrays from feature importance and feature names
  feature_importance = np.array(importance)
  feature_names = np.array(names)

  #Create a DataFrame using a Dictionary
  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)

  #Sort the DataFrame in order decreasing feature importance
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  #Define size of bar plot
  plt.figure(figsize=(20,20))
  #Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  #Add chart labels
  plt.title(model_type + 'FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')

plot_feature_importance(xg_reg.feature_importances_,x_train.columns,'XG BOOST')